<h3>SQL Script<h3>

<h3>ข้อ 1</h3>
<label>แสดงข้อมูล ผู้กำกับ (director) ที่ได้รับ ความนิยม (rating) สูงที่สุดของแต่ละประเภทหนัง
(movie_genres)
</label>

In [ ]:
SELECT dir_fname, dir_lname, gen_title
FROM Movie
JOIN movie_direction USING(mov_id)
JOIN director USING(dir_id)
NATURAL JOIN rating
NATURAL JOIN movie_genres
NATURAL JOIN genres
GROUP BY dir_fname, dir_lname, gen_title
HAVING MAX(rev_stars) = MAX(rev_stars);

<h3>ข้อ 2</h3>
<label>แสดงข้อมูล นักแสดงแต่ละคนแสดงหนังแต่ละเรื่องรวมเป็นระยะเวลาเท่าใด โดยคิดเฉพาะหนังที่
คะแนนความนิยมไม่ใช่ null
<label>

In [ ]:
SELECT actor.act_fname, actor.act_lname,actor.act_gender,SUM(movie.mov_time) AS total_time
FROM movie
JOIN movie_cast USING(mov_id)
JOIN actor USING(act_id)
LEFT JOIN rating USING(mov_id)
WHERE rating.rev_stars IS NOT NULL
GROUP BY actor.act_fname, actor.act_lname,actor.act_gender;

<h3>ข้อ 3</h3>
<label>แสดงข้อมูล จำนวนครั้งที่ ผู้กำกับ (director) และนักแสดงหญิง (actor) เคยร่วมงานกัน โดยแสดง
เฉพาะ 5 ลำดับแรกของนักแสดงหญิงที่ร่วมงานกับผู้กำกับคนนั้นบ่อยที่สุด

<label>

In [ ]:
SELECT director.dir_fname,director.dir_lname,actor.act_fname,actor.act_lname,COUNT(*) AS collaboration_count
FROM movie
JOIN movie_cast ON movie.mov_id = movie_cast.mov_id
JOIN actor ON movie_cast.act_id = actor.act_id
JOIN movie_direction ON movie.mov_id = movie_direction.mov_id
JOIN director ON movie_direction.dir_id = director.dir_id
WHERE actor.act_gender = 'F'
GROUP BY director.dir_id, actor.act_id
ORDER BY collaboration_count DESC
LIMIT 5;

<h3>ข้อ 4</h3>
<label> เปลี่ยนประเภทข้อมูล gen_title จาก char (20) เป็น integer โดยเรียงตามตัวอักษร เช่น action
แปลงเป็น 1, animation แปลงเป็น 2, biography แปลงเป็น 3, …
<label>

In [ ]:
ALTER TABLE genres
ADD COLUMN gen_id_temp INTEGER;

UPDATE genres
SET gen_id_temp = ROW_NUMBER() OVER (ORDER BY gen_title);

ALTER TABLE genres
DROP COLUMN gen_id;

ALTER TABLE genres
RENAME COLUMN gen_id_temp TO gen_id;

<h3>ข้อ 5</h3>
<label> แก้ไขเพศของนักแสดง ที่ชื่อขึ้นต้นด้วยคำว่า Em, Char หรือ ชื่อลงท้ายด้วยคำว่า dy, sy, lia เป็น
เพศหญิง โดยไม่สนใจตัวเล็กตัวใหญ่
<label>

In [ ]:
UPDATE actor
SET act_gender = 'F'
WHERE 
  UPPER(act_fname) LIKE 'EM%'
  OR UPPER(act_fname) LIKE 'CHAR%'
  OR UPPER(act_lname) LIKE '%DY'
  OR UPPER(act_lname) LIKE '%SY'
  OR UPPER(act_lname) LIKE '%LIA';